![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# LLM Session Memory

Large Language Models are inherently stateless and have no knowledge of previous interactions with a user, or even of previous parts of the current conversation. While this may not be noticeable when asking simple questions, it becomes a hinderance when engaging in long running conversations that rely on conversational context.

The solution to this problem is to append the previous conversation history to each subsequent call to the LLM.

This notebook will show how to use Redis to structure and store and retrieve this conversational session memory.

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/session-memory/session_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Environment setup

## set cohere api key

In [1]:
import os
import getpass


if "COHERE_API_KEY" not in os.environ:
    os.environ["COHERE_API_KEY"] = getpass.getpass("COHERE_API_KEY")

## Run local redis (for colab)
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

## For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

## Define the Redis Connection URL and Client

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [2]:
import os
from redis import Redis

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

redis_client = Redis.from_url(REDIS_URL)

## Init CohereClient as LLM layer

In [3]:
from typing import Dict, List
import cohere
import os

class CohereClient():
    def __init__(self, api_key: str = None, model: str = 'command-r-plus'):
        api_key = api_key or os.getenv("COHERE_API_KEY")
        self.client = cohere.Client(api_key)
        self._model = model

    def converse(self, prompt: str, context: List[Dict]) -> str:
        context = self.remap(context)
        response = self.client.chat(
                model=self._model,
                chat_history = context,
                message=prompt,
                )
        return response.text

    def remap(self, context) -> List[Dict]:
        ''' re-index the chat history to match the Cohere API requirements '''
        new_context = []
        for statement in context:
            if statement["role"] == "user":
                new_statement = {"role": "USER", "message": statement["content"]}
            elif statement["role"] == "llm":
                new_statement = {"role": "CHATBOT", "message": statement["content"]}
            elif statement["role"] == "system":
                new_statement = {"role": "SYSTEM", "message": statement["content"]}
            else:
                raise ValueError(f'Unknown chat role {statement["role"]}')
            new_context.append(new_statement)
        return new_context

client = CohereClient()

## Import SemanticSessionManager

redisvl provides the SemanticSessionManager for easy management of session state.

In [4]:
from redisvl.extensions.session_manager import SemanticSessionManager

user_session = SemanticSessionManager(name="llm chef")
user_session.add_message({"role":"system", "content":"You are a helpful chef, assisting people in making delicious meals"})

On each call to the LLM we can first retrieve the recent history of the conversation. On the first call it will just be the preamble we set.

In [5]:
prompt = "can you give me some ideas for breakfast?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me some ideas for breakfast?

LLM:  Sure! Here are a few breakfast ideas:

- Avocado toast: This is a quick and easy option that's both delicious and nutritious. Simply toast some bread, smash an avocado on top, and add a squeeze of lemon or lime juice and a pinch of salt and pepper to taste. You can also add additional toppings like tomato, feta cheese, or a fried egg.
- Oatmeal: Oatmeal is a hearty and healthy breakfast option that can be prepared in a variety of ways. You can cook it on the stovetop or in the microwave, and top it with fruits, nuts, seeds, or a drizzle of honey.
- Smoothie bowl: Blend together your favorite fruits, vegetables, and liquid base (such as juice, milk, or yogurt) to make a smoothie, and then pour it into a bowl. Top it with granola, nuts, seeds, or fresh fruit for a beautiful and tasty breakfast.
- Eggs: There are endless ways to prepare eggs for breakfast. You can scramble them, fry them, poach them, or make an omelet with your favor

Let's continue this conversation and inspect the context we're passing to the LLM as we go.

In [6]:
prompt = "can you give me the recipe for those pancakes?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me the recipe for those pancakes?

LLM:  Sure, here's a basic recipe for fluffy pancakes:

Ingredients:

- 2 cups all-purpose flour
- 2 teaspoons baking powder
- 1/2 teaspoon salt
- 2 tablespoons sugar
- 2 cups milk
- 2 eggs
- 2 tablespoons melted butter, plus more for the pan
- 1 teaspoon vanilla extract (optional)

Instructions:

1. In a large bowl, whisk together the flour, baking powder, salt, and sugar until well combined.

2. In a separate bowl, whisk together the milk, eggs, melted butter, and vanilla extract (if using).

3. Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter; a few lumps are okay.

4. Heat a nonstick pan or griddle over medium heat and grease it with a small amount of butter.

5. Pour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are small bubbles on the surface, about 2-3 minutes.

6. Flip the pancakes carefully and cook for an 

In [7]:
for ctx in context:
    print(ctx)

{'role': 'system', 'content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': 'user', 'content': 'can you give me some ideas for breakfast?'}
{'role': 'llm', 'content': "Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a quick and easy option that's both delicious and nutritious. Simply toast some bread, smash an avocado on top, and add a squeeze of lemon or lime juice and a pinch of salt and pepper to taste. You can also add additional toppings like tomato, feta cheese, or a fried egg.\n- Oatmeal: Oatmeal is a hearty and healthy breakfast option that can be prepared in a variety of ways. You can cook it on the stovetop or in the microwave, and top it with fruits, nuts, seeds, or a drizzle of honey.\n- Smoothie bowl: Blend together your favorite fruits, vegetables, and liquid base (such as juice, milk, or yogurt) to make a smoothie, and then pour it into a bowl. Top it with granola, nuts, seeds, or fresh fruit for a beautiful and tasty br

In [8]:
prompt ="I am vegetarian. Can you remove the eggs?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I am vegetarian. Can you remove the eggs?

LLM:  Absolutely! Here's an egg-free version of the fluffy pancake recipe:

Ingredients:

- 2 cups all-purpose flour
- 2 teaspoons baking powder
- 1/2 teaspoon salt
- 2 tablespoons sugar
- 2 cups milk (dairy or non-dairy alternative)
- 2 tablespoons melted butter, plus more for the pan
- 1 teaspoon vanilla extract (optional)

Instructions:

1. In a large bowl, whisk together the flour, baking powder, salt, and sugar until well combined.

2. In a separate bowl, whisk together the milk, melted butter, and vanilla extract (if using).

3. Pour the wet ingredients into the dry ingredients and stir until just combined. Again, be careful not to overmix the batter; a few lumps are okay.

4. Heat a nonstick pan or griddle over medium heat and grease it with a small amount of butter.

5. Pour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are small bubbles on the surface, about 2-3 minutes.

6. Care

In [9]:
for ctx in context:
    print(ctx)

{'role': 'system', 'content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': 'user', 'content': 'can you give me some ideas for breakfast?'}
{'role': 'llm', 'content': "Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a quick and easy option that's both delicious and nutritious. Simply toast some bread, smash an avocado on top, and add a squeeze of lemon or lime juice and a pinch of salt and pepper to taste. You can also add additional toppings like tomato, feta cheese, or a fried egg.\n- Oatmeal: Oatmeal is a hearty and healthy breakfast option that can be prepared in a variety of ways. You can cook it on the stovetop or in the microwave, and top it with fruits, nuts, seeds, or a drizzle of honey.\n- Smoothie bowl: Blend together your favorite fruits, vegetables, and liquid base (such as juice, milk, or yogurt) to make a smoothie, and then pour it into a bowl. Top it with granola, nuts, seeds, or fresh fruit for a beautiful and tasty br

In [10]:
prompt = "I am also vegan. Can you replace the butter too?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I am also vegan. Can you replace the butter too?

LLM:  Sure, to make the recipe completely vegan, you can replace the butter with a vegan alternative. Here's the updated recipe:

Ingredients:

- 2 cups all-purpose flour
- 2 teaspoons baking powder
- 1/2 teaspoon salt
- 2 tablespoons sugar
- 2 cups plant-based milk (such as soy, almond, or oat milk)
- 2 tablespoons neutral-flavored oil (such as canola or vegetable oil), plus more for the pan
- 1 teaspoon vanilla extract (optional)

Instructions:

1. In a large bowl, whisk together the flour, baking powder, salt, and sugar until well combined.

2. In a separate bowl, whisk together the plant-based milk, oil, and vanilla extract (if using).

3. Pour the wet ingredients into the dry ingredients and stir until just combined. It's important not to overmix the batter; a few lumps are totally fine.

4. Heat a nonstick pan or griddle over medium heat and lightly grease it with a small amount of oil.

5. Pour about 1/4 cup of batter onto

In [11]:
for ctx in context:
    print(ctx)

{'role': 'llm', 'content': "Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a quick and easy option that's both delicious and nutritious. Simply toast some bread, smash an avocado on top, and add a squeeze of lemon or lime juice and a pinch of salt and pepper to taste. You can also add additional toppings like tomato, feta cheese, or a fried egg.\n- Oatmeal: Oatmeal is a hearty and healthy breakfast option that can be prepared in a variety of ways. You can cook it on the stovetop or in the microwave, and top it with fruits, nuts, seeds, or a drizzle of honey.\n- Smoothie bowl: Blend together your favorite fruits, vegetables, and liquid base (such as juice, milk, or yogurt) to make a smoothie, and then pour it into a bowl. Top it with granola, nuts, seeds, or fresh fruit for a beautiful and tasty breakfast.\n- Eggs: There are endless ways to prepare eggs for breakfast. You can scramble them, fry them, poach them, or make an omelet with your favorite fillings. Serve the

This works, but our context keeps growing, increasing our LLM token count, which increases latency and cost.
Conversation histories can be truncated, but that can break quickly.

In [12]:
prompt = "I changed my mind. Can you give me the first recipe from your list?"
context = user_session.get_recent(top_k=5)
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I changed my mind. Can you give me the first recipe from your list?

LLM:  Sure, here's the first recipe for fluffy pancakes that I provided:

Ingredients:

- 2 cups all-purpose flour
- 2 teaspoons baking powder
- 1/2 teaspoon salt
- 2 tablespoons sugar
- 2 cups milk
- 2 eggs
- 2 tablespoons melted butter, plus more for the pan
- 1 teaspoon vanilla extract (optional)

Instructions:

1. In a large bowl, whisk together the flour, baking powder, salt, and sugar until well combined.

2. In a separate bowl, whisk together the milk, eggs, melted butter, and vanilla extract (if using).

3. Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter; a few lumps are okay and will ensure your pancakes are nice and fluffy.

4. Heat a nonstick pan or griddle over medium heat. Grease it with a small amount of butter to prevent the pancakes from sticking.

5. Pour about 1/4 cup of batter onto the pan for each pancake. Cook the pancakes

In [13]:
for ctx in context:
    print(ctx)

{'role': 'llm', 'content': "Sure, here's a basic recipe for fluffy pancakes:\n\nIngredients:\n\n- 2 cups all-purpose flour\n- 2 teaspoons baking powder\n- 1/2 teaspoon salt\n- 2 tablespoons sugar\n- 2 cups milk\n- 2 eggs\n- 2 tablespoons melted butter, plus more for the pan\n- 1 teaspoon vanilla extract (optional)\n\nInstructions:\n\n1. In a large bowl, whisk together the flour, baking powder, salt, and sugar until well combined.\n\n2. In a separate bowl, whisk together the milk, eggs, melted butter, and vanilla extract (if using).\n\n3. Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter; a few lumps are okay.\n\n4. Heat a nonstick pan or griddle over medium heat and grease it with a small amount of butter.\n\n5. Pour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are small bubbles on the surface, about 2-3 minutes.\n\n6. Flip the pancakes carefully and cook for an addition

The LLM has forgotten about the avocado toast it mentioned because that was too far back in it's conversation history!

## Semantic session memory

Rather than naively pass the most recent conversation we can use RedisVL to find only the semantically relevant sections of our conversation based on our new prompt. We can do this with the `fetch_relevant()` method.

In [14]:
prompt = "Can you give me the avocado one?"
user_session.set_distance_threshold(0.75)
context = user_session.get_relevant(prompt=prompt)
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  Can you give me the avocado one?

LLM:  Certainly! Here's a simple recipe for avocado toast:

Ingredients:
- 1 ripe avocado
- 2 slices of your favorite bread
- 1/2 lemon or lime
- Salt and pepper to taste
- Optional toppings: crushed red pepper flakes, chopped tomato, feta cheese, or a fried egg

Directions:

1. Toast the bread to your desired level.
2. While the bread is toasting, cut the avocado in half, remove the pit, and scoop the flesh into a small bowl.
3. Using a fork, mash the avocado until it reaches your desired consistency. You can leave it chunkier or make it smoother, depending on your preference.
4. Add a squeeze of lemon or lime juice to the mashed avocado. This will help to brighten the flavor and prevent the avocado from browning too quickly.
5. Add a pinch of salt and pepper to taste.
6. Spread the avocado mixture onto the toasted bread.
7. If desired, add additional toppings such as crushed red pepper flakes for a spicy kick, chopped tomato for freshness, fet

In [15]:
for ctx in context:
    print(ctx)

{'role': 'llm', 'content': "Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a quick and easy option that's both delicious and nutritious. Simply toast some bread, smash an avocado on top, and add a squeeze of lemon or lime juice and a pinch of salt and pepper to taste. You can also add additional toppings like tomato, feta cheese, or a fried egg.\n- Oatmeal: Oatmeal is a hearty and healthy breakfast option that can be prepared in a variety of ways. You can cook it on the stovetop or in the microwave, and top it with fruits, nuts, seeds, or a drizzle of honey.\n- Smoothie bowl: Blend together your favorite fruits, vegetables, and liquid base (such as juice, milk, or yogurt) to make a smoothie, and then pour it into a bowl. Top it with granola, nuts, seeds, or fresh fruit for a beautiful and tasty breakfast.\n- Eggs: There are endless ways to prepare eggs for breakfast. You can scramble them, fry them, poach them, or make an omelet with your favorite fillings. Serve the

Looking at the context we see that only the conversation section that had to do with avocados was retrieved. We don't need to know about pancakes to make avocado toast.

In [16]:
user_session.clear()